In [104]:
import numpy as np
import pandas as pd  
from IPython.display import display 
import subprocess
import itertools
import warnings
import os 
import rpv_definitions as rdef
import rpv_misc as rmisc
import abcrpv as rpv

In [25]:

def signature_ordering(instr):
    """
    Make signatures in ordered form based on rdef.sig_order
    @TODO SF bracket treatment

    Inputs:
    ------
    -instr     (str): signature in one-char syntax. Should not have SF brackets (refer same_flavour function) 

    """
    assert type(instr) == str, "Input must be a string!"
    fixedinstr = instr
    nstr = len(instr)
    outstr = ""
    
    # If () in string, take it out and put it in front
    # Could be done with re, but annoying to learn re syntax (@TODO)
    if "(" in instr and ")" in instr :
        sfdat = ""
        elsedat = ""
        boolget = False
        for i in instr:
            if i == "(":
                boolget = True

            if boolget == True:
                sfdat = sfdat + i
            else:
                elsedat = elsedat + i

            if i == ")":
                boolget = False

        outstr = outstr+sfdat

    else:
        elsedat = instr

    #Reordering
    for i in rdef.sig_order:
        for j in elsedat:
            if j == i:
                outstr = outstr + i
    
    #String Length should not change by reordering, unless non-signature-syntax char are in string (If so print out)
    if len(outstr) != nstr:
        raise Warning(
            "String length changed! \n Input = {f} or {i} \n Output= {o}".format(f=fixedinstr,i=instr,o=outstr))
    
    #Two MET is just MET
    while outstr.count("E") > 1 :
        outstr = outstr.replace("EE", "E",100)
        outstr = signature_ordering(outstr)
    
    return outstr

In [18]:
def set_elements_simplified(instr):
    """
    In terms of signatures:
        Get all possible signatures from signatures with simplified objects (J,3,L)
    or in terms of sets:
        Get all elements from signature set defined via simplified objects (J,3,L)

    @TODO SF bracket treament

    Inputs
    ------
    -instr  (str): input signature in one-char syntax  

    Example
    ------
    JJjtLTEE => ['tjjjTlE', 'tjjjTTE', 'ttjjTlE', 'ttjjTTE', 'tbjjTlE', 'tbjjTTE', 'ttjjTlE', 'ttjjTTE', 'tttjTlE', 'tttjTTE', 'ttbjTlE', 'ttbjTTE', 'tbjjTlE', 'tbjjTTE', 'ttbjTlE', 'ttbjTTE', 'tbbjTlE', 'tbbjTTE']
    """
    strlist = list(instr)
    expand_list = []
    for i in strlist:
        if i == "J":
            expand_list.append(["j","t","b"])
        elif i == "3":
            expand_list.append(["t","b"])
        elif i == "L":
            expand_list.append(["l","T"])
        else:
            expand_list.append(i)
    return [signature_ordering("".join(x)) for x in list(itertools.product(*expand_list))]

In [ ]:

def get_all_superset(instr,optJ,opt3,optL):
    """
    In terms of signatures:
        Get all possible simplified signatures that can lead to input signature 
    or in terms of sets:
        Get all possible superset (defined via simplified objects (J,3,L)) that contain instr
    @TODO SF bracket treament

    Inputs
    ------
    -instr  (str) : input signature in one-char syntax  
    -optX   (bool): Flag to look up simplified signatures with X

    Example
    ------
    Input = tjTlE
      optJ   opt3    optL
      True   True    True  ['JJLLE', 'JJLlE', 'JJLTE', 'JJTlE', 'JjLLE', 'JjLlE', 'JjLTE', 'JjTlE', 'J3LLE', 'J3LlE', 'J3LTE', 'J3TlE', '3jLLE', '3jLlE', '3jLTE', '3jTlE', 'JtLLE', 'JtLlE', 'JtLTE', 'JtTlE', 'tjLLE', 'tjLlE', 'tjLTE', 'tjTlE']
      True   False   False ['JJTlE', 'JjTlE', 'JtTlE', 'tjTlE']
      False  True    False ['3jTlE', 'tjTlE']
      False  False   True  ['tjLLE', 'tjLlE', 'tjLTE', 'tjTlE']
      True   True    False ['JJTlE', 'JjTlE', 'J3TlE', '3jTlE', 'JtTlE', 'tjTlE']
      True   False   True  ['JJLLE', 'JJLlE', 'JJLTE', 'JJTlE', 'JjLLE', 'JjLlE', 'JjLTE', 'JjTlE', 'JtLLE', 'JtLlE', 'JtLTE', 'JtTlE', 'tjLLE', 'tjLlE', 'tjLTE', 'tjTlE']
      False  True    True  ['3jLLE', '3jLlE', '3jLTE', '3jTlE', 'tjLLE', 'tjLlE', 'tjLTE', 'tjTlE']
    """
    strlist = list(instr)
    expand_list = []
    for i in strlist:
        if i == "j" and optJ==True:
            expand_list.append(["J",i])
        elif i == "t" or i == "b":
            if optJ==True and opt3==True:
                expand_list.append(["J","3",i])
            elif optJ==True:
                expand_list.append(["J",i])
            elif opt3==True:
                expand_list.append(["3",i])
            else:
                expand_list.append(i)    
        elif i == "l" and optL==True :
            expand_list.append(["L",i])
        elif i == "T" and optL==True:
            expand_list.append(["L",i])
        else:
            expand_list.append(i)
    return [signature_ordering("".join(x)) for x in list(itertools.product(*expand_list))]


In [ ]:

def to_J(inx):
    if type(inx) != str:
        return "input not string!"
    else:
        inx = inx.replace("j", "J")
        inx = inx.replace("t", "J")
        inx = inx.replace("b", "J")
        inx = inx.replace("3", "J")
    return inx
    
def to_j3(inx):
    if type(inx) != str:
        return "input not string!"
    else:
        inx = inx.replace("t", "3")
        inx = inx.replace("b", "3")
        return inx
    
def to_L(inx):
    if type(inx) != str:
        return "input not string!"
    else:
        inx = inx.replace("l","L")
        inx = inx.replace("T","L")
        return inx